In [11]:
import requests
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize

- DART 오픈 API 사용법
- json_normalize() 활용
- 기업개황 api
- 정보검색 api
- 개별보고서와 하위 보고서 url 얻기
- 첨부 파일과 재무제표 엑셀 파일 가져오기

##### 요청 주소
- http://dart.fss.or.kr/api/search.xml?auth=xxx     xml 응답
- http://dart.fss.or.kr/api/search.json?auth=xxx    json 응답

### 요청 변수
- `auth`	발급받은 인증키(40자리)(필수)
- `crp_cd`	공시대상회사의 종목코드(상장사:숫자 6자리) 또는 고유번호(기타법인:숫자 8자리)
- `end_dt`	검색종료 접수일자(YYYYMMDD) : 없으면 당일
- `start_dt`	검색시작 접수일자(YYYYMMDD) : 없으면 end_dt
- crp_cd가 없는 경우 검색기간은 3개월로 제한
- `fin_rpt`	최종보고서만 검색여부(Y or N) 기본값 : N (정정이 있는 경우 최종정정만 검색)
- `dsp_tp`	정기공시(A), 주요사항보고(B), 발행공시(C), 지분공시(D), 기타공시(E),외부감사관련(F), 펀드공시(G), 자산유동화(H), 거래소공시(I), 공정위공시(J)
- `bsn_tp`	정기공시(5), 주요사항보고(3), 발행공시(11), 지분공시(4), 기타공시(9),외부감사관련(3), 펀드공시(3), 자산유동화(6), 거래소공시(6), 공정위공시(5)(상세 유형 참조)
- `sort`	접수일자(date), 회사명(crp), 보고서명(rpt) 기본값 : date
- `series`	오름차순(asc), 내림차순(desc) 기본값 : desc
- `page_no`	페이지 번호(1~n) 기본값 : 1
- `page_set`	페이지당 건수(1~100) 기본값 : 10, 최대값 : 100
- `callback`	콜백함수명(JSONP용)

#### 에러 코드
- `000`	정상
- `010`	등록되지 않은 키입니다.
- `011`	사용할 수 없는 키입니다. 오픈API에 등록되었으나, 일시적으로 사용 중지된 키를 -통하여 검색하는 경우 발생합니다.
- `020`	요청 제한을 초과하였습니다. 일반적으로는 10,000건 이상의 요청에 대하여 이 에러 메시지가 발생되나, 요청 제한이 다르게 설정된 경우에는 이에 준하여 발생됩니다.
- `100`	필드의 부적절한 값입니다. 필드 설명에 없는 값을 사용한 경우에 발생하는 메시지입니다.
- `900`	정의되지 않은 오류가 발생하였습니다.

### 요청 URL 예시
http://dart.fss.or.kr/api/search.json `auth=e81e78aa519d1a1c4ec7867ef12a937ab6cbd4c7`&`crp_cd=005930`&`start_dt=20000101`&`end_dt=20140730`&`dsp_tp=A`&`sort=date`

- 발급받은 인증키를 사용해서 받은 json 자료의 일부이다.

{"err_code":"000","err_msg":"정상","page_no":1,"page_set":10,"total_count":61,"total_page":7,"list":[{"crp_cls":"Y","crp_nm":"삼성전자","crp_cd":"005930","rpt_nm":"[첨부추가]사업보고서 (1999.12)","rcp_no":"20000330000796","flr_nm":"삼성전자","rcp_dt":"20000330","rmk":"연"},{"crp_cls":"Y","crp_nm":"삼성전자","crp_cd":"005930","rpt_nm":"분기보고서 (2000.03)","rcp_no":"20000515000739","flr_nm":"삼성전자","rcp_dt":"20000515","rmk":""},{"crp_cls":"Y","crp_nm":"삼성전자",

In [12]:
#실제 인증키를 사용
auth = '***'
headers={'Referer':'https://dart.fss.or.kr/dsap001/guide.do'}
#헤더 : 실제 인증키를 사용시 사용하지 않아도 된다.

XML 형식 요청하기

In [13]:
import requests

url_tmpl = 'http://dart.fss.or.kr/api/search.xml?auth={auth}'
url = url_tmpl.format(auth=auth)

r = requests.get(url)
r.text

'<?xml version="1.0" encoding="utf-8"?>\n<result>\n  <err_code>000</err_code>\n  <err_msg>정상</err_msg>\n  <page_no>1</page_no>\n  <page_set>10</page_set>\n  <total_count>104</total_count>\n  <total_page>11</total_page>\n  <list>\n    <crp_cls>Y</crp_cls>\n    <crp_nm>동성제약</crp_nm>\n    <crp_cd>002210</crp_cd>\n    <rpt_nm>전환가액ㆍ신주인수권행사가액ㆍ교환가액의조정(안내공시)</rpt_nm>\n    <rcp_no>20180618800164</rcp_no>\n    <flr_nm>동성제약</flr_nm>\n    <rcp_dt>20180618</rcp_dt>\n    <rmk>유</rmk>\n  </list>\n  <list>\n    <crp_cls>Y</crp_cls>\n    <crp_nm>명문제약</crp_nm>\n    <crp_cd>017180</crp_cd>\n    <rpt_nm>[기재정정]사업보고서 (2017.12)</rpt_nm>\n    <rcp_no>20180618000104</rcp_no>\n    <flr_nm>명문제약</flr_nm>\n    <rcp_dt>20180618</rcp_dt>\n    <rmk>연</rmk>\n  </list>\n  <list>\n    <crp_cls>Y</crp_cls>\n    <crp_nm>현대중공업지주</crp_nm>\n    <crp_cd>267250</crp_cd>\n    <rpt_nm>임원ㆍ주요주주특정증권등소유상황보고서</rpt_nm>\n    <rcp_no>20180618000103</rcp_no>\n    <flr_nm>윤중근</flr_nm>\n    <rcp_dt>20180618</rcp_dt>\n    <rmk></rmk>\n  </l

JSON 형식 요청하기

In [14]:
import requests

url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}'
url = url_tmpl.format(auth=auth)

r = requests.get(url)
r.text

'{"err_code":"000","err_msg":"정상","page_no":1,"page_set":10,"total_count":104,"total_page":11,"list":[{"crp_cls":"Y","crp_nm":"동성제약","crp_cd":"002210","rpt_nm":"전환가액ㆍ신주인수권행사가액ㆍ교환가액의조정(안내공시)","rcp_no":"20180618800164","flr_nm":"동성제약","rcp_dt":"20180618","rmk":"유"},{"crp_cls":"Y","crp_nm":"명문제약","crp_cd":"017180","rpt_nm":"[기재정정]사업보고서 (2017.12)","rcp_no":"20180618000104","flr_nm":"명문제약","rcp_dt":"20180618","rmk":"연"},{"crp_cls":"Y","crp_nm":"현대중공업지주","crp_cd":"267250","rpt_nm":"임원ㆍ주요주주특정증권등소유상황보고서","rcp_no":"20180618000103","flr_nm":"윤중근","rcp_dt":"20180618","rmk":""},{"crp_cls":"Y","crp_nm":"현대중공업지주","crp_cd":"267250","rpt_nm":"최대주주등소유주식변동신고서","rcp_no":"20180618800161","flr_nm":"현대중공업지주","rcp_dt":"20180618","rmk":"유"},{"crp_cls":"K","crp_nm":"휴온스","crp_cd":"243070","rpt_nm":"기업설명회(IR)개최","rcp_no":"20180618900160","flr_nm":"휴온스","rcp_dt":"20180618","rmk":"코"},{"crp_cls":"Y","crp_nm":"LS","crp_cd":"006260","rpt_nm":"타법인주식및출자증권취득결정(자회사의 주요경영사항)","rcp_no":"20180618800159","flr_nm":"LS","rcp

### json_normalize()
- JSON 데이터를 손쉽게 테이블 형태로 만들어줍니다.

In [15]:
from pandas.io.json import json_normalize
import json

json_str = '''
[
    {"A": 1, "B": 2, "C": 3},
    {"A": 4, "B": 5, "C": 6},
    {"A": 7, "B": 8, "C": 9}
]'''
json_str

'\n[\n    {"A": 1, "B": 2, "C": 3},\n    {"A": 4, "B": 5, "C": 6},\n    {"A": 7, "B": 8, "C": 9}\n]'

In [16]:
jo = json.loads(json_str)#deserialize data 
jo

[{'A': 1, 'B': 2, 'C': 3}, {'A': 4, 'B': 5, 'C': 6}, {'A': 7, 'B': 8, 'C': 9}]

In [17]:
result = json_normalize(jo)#"Normalize" semi-structured JSON data into a flat table
result

,A,B,C
0,1,2,3
1,4,5,6
2,7,8,9


기업 개황 API
- 해당 기업의 코드를 더 해줍니다

In [18]:
url_tmpl = 'http://dart.fss.or.kr/api/company.json?auth={auth}&crp_cd={code}'
auth = '***'
code = '005930'#삼성전자
url = url_tmpl.format(auth=auth,code=code)
r=requests.get(url)
jo = json.loads(r.text)
jo

{'acc_mt': '12',
 'adr': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'bsn_no': '1248100998',
 'ceo_nm': '김기남, 김현석, 고동진',
 'crp_cls': 'Y',
 'crp_nm': '삼성전자(주)',
 'crp_nm_e': 'SAMSUNG ELECTRONICS CO,.LTD',
 'crp_nm_i': '삼성전자',
 'crp_no': '1301110006246',
 'err_code': '000',
 'err_msg': '정상',
 'est_dt': '19690113',
 'fax_no': '031-200-7538',
 'hm_url': 'www.sec.co.kr',
 'ind_cd': '264',
 'ir_url': '',
 'phn_no': '031-200-1114',
 'stock_cd': '005930'}

#### 각 필드의 의미
- err_code: 에러코드(오류 메시지 참조)
- err_msg: 에러메시지(오류 메시지 참조)
- crp_nm: 정식명칭
- crp_nm_e: 영문명칭
- crp_nm_i: 종목명(상장사) 또는 약식명칭(기타법인)
- stock_cd: 상장회사인 경우 주식의 종목코드
- ceo_nm: 대표자명
- crp_cls: 법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)
- crp_no: 법인등록번호
- bsn_no: 사업자등록번호
- adr: 주소
- hm_url: 홈페이지
- ir_url: IR홈페이지
- phn_no: 전화번호
- fax_no: 팩스번호
- ind_cd: 업종코드
- est_dt: 설립일(YYYYMMDD)
- acc_mt: 결산월(MM)

In [19]:
jo['adr']

'경기도 수원시 영통구  삼성로 129 (매탄동)'

정보 검색 API
- 해당 기업의 코드 삽입
- start_df
- bsn_tp

#### 검색 API 샘플
- 당일 접수 10건
http://dart.fss.or.kr/api/search.xml?auth=xxx
- 당일 접수 100건
http://dart.fss.or.kr/api/search.xml?auth=xxx&page_set=100
- 회사의 당일 접수 10건
http://dart.fss.or.kr/api/search.xml?auth=xxx&crp_cd=xxx
- 회사의 당일 접수 최종보고서만 10건
http://dart.fss.or.kr/api/search.xml?auth=xxx&crp_cd=xxx&fin_rpt=Y
- 회사의 전체(19990101~당일) 공시 10건
http://dart.fss.or.kr/api/search.xml?auth=xxx&crp_cd=xxx&start_dt=19990101
- 회사의 전체 정기공시 10건
http://dart.fss.or.kr/api/search.xml?auth=xxx&crp_cd=xxx&start_dt=19990101&dsp_tp=A
- 회사의 전체 사업+반기+분기보고서 10건
http://dart.fss.or.kr/api/search.xml?auth=xxx&crp_cd=xxx&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003

In [20]:
#당일 접수 10건
url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}'
auth = '***'
code = '005930'
url = url_tmpl.format(auth=auth)
r = requests.get(url)
jo = json.loads(r.text)
result=json_normalize(jo,'list')
result

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,002210,Y,동성제약,동성제약,20180618,20180618800164,유,전환가액ㆍ신주인수권행사가액ㆍ교환가액의조정(안내공시)
1,017180,Y,명문제약,명문제약,20180618,20180618000104,연,[기재정정]사업보고서 (2017.12)
2,267250,Y,현대중공업지주,윤중근,20180618,20180618000103,,임원ㆍ주요주주특정증권등소유상황보고서
3,267250,Y,현대중공업지주,현대중공업지주,20180618,20180618800161,유,최대주주등소유주식변동신고서
4,243070,K,휴온스,휴온스,20180618,20180618900160,코,기업설명회(IR)개최
5,006260,Y,LS,LS,20180618,20180618800159,유,타법인주식및출자증권취득결정(자회사의 주요경영사항)
6,005940,Y,NH투자증권,NH투자증권,20180618,20180618000096,,증권발행실적보고서
7,032800,K,판타지오,판타지오,20180618,20180618900155,코,임시주주총회결과
8,016360,Y,삼성증권,삼성증권,20180618,20180618000094,,증권발행실적보고서
9,056730,K,포스링크,포스링크,20180618,20180618900153,코,전환가액의조정(제16회차)


In [21]:
jo

{'err_code': '000',
 'err_msg': '정상',
 'list': [{'crp_cd': '002210',
   'crp_cls': 'Y',
   'crp_nm': '동성제약',
   'flr_nm': '동성제약',
   'rcp_dt': '20180618',
   'rcp_no': '20180618800164',
   'rmk': '유',
   'rpt_nm': '전환가액ㆍ신주인수권행사가액ㆍ교환가액의조정(안내공시)'},
  {'crp_cd': '017180',
   'crp_cls': 'Y',
   'crp_nm': '명문제약',
   'flr_nm': '명문제약',
   'rcp_dt': '20180618',
   'rcp_no': '20180618000104',
   'rmk': '연',
   'rpt_nm': '[기재정정]사업보고서 (2017.12)'},
  {'crp_cd': '267250',
   'crp_cls': 'Y',
   'crp_nm': '현대중공업지주',
   'flr_nm': '윤중근',
   'rcp_dt': '20180618',
   'rcp_no': '20180618000103',
   'rmk': '',
   'rpt_nm': '임원ㆍ주요주주특정증권등소유상황보고서'},
  {'crp_cd': '267250',
   'crp_cls': 'Y',
   'crp_nm': '현대중공업지주',
   'flr_nm': '현대중공업지주',
   'rcp_dt': '20180618',
   'rcp_no': '20180618800161',
   'rmk': '유',
   'rpt_nm': '최대주주등소유주식변동신고서'},
  {'crp_cd': '243070',
   'crp_cls': 'K',
   'crp_nm': '휴온스',
   'flr_nm': '휴온스',
   'rcp_dt': '20180618',
   'rcp_no': '20180618900160',
   'rmk': '코',
   'rpt_nm': '기업설명회(

In [22]:
# 당일 접수 100건 (최대 100건)
url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}&page_set=100'
url = url_tmpl.format(auth=auth)
r = requests.get(url)
jo = json.loads(r.text)
result = json_normalize(jo, 'list')
result.head()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,002210,Y,동성제약,동성제약,20180618,20180618800164,유,전환가액ㆍ신주인수권행사가액ㆍ교환가액의조정(안내공시)
1,017180,Y,명문제약,명문제약,20180618,20180618000104,연,[기재정정]사업보고서 (2017.12)
2,267250,Y,현대중공업지주,윤중근,20180618,20180618000103,,임원ㆍ주요주주특정증권등소유상황보고서
3,267250,Y,현대중공업지주,현대중공업지주,20180618,20180618800161,유,최대주주등소유주식변동신고서
4,243070,K,휴온스,휴온스,20180618,20180618900160,코,기업설명회(IR)개최


In [23]:
# 회사의 전체 사업+반기+분기보고서 100건
url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003&page_set=100'
auth = '***'
code = '005930'
url = url_tmpl.format(auth=auth,code=code)
r = requests.get(url)
jo = json.loads(r.text)
result = json_normalize(jo, 'list')
result.head()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,005930,Y,삼성전자,삼성전자,20180515,20180515001699,,분기보고서 (2018.03)
1,005930,Y,삼성전자,삼성전자,20180402,20180402005019,연,사업보고서 (2017.12)
2,005930,Y,삼성전자,삼성전자,20171114,20171114001694,,분기보고서 (2017.09)
3,005930,Y,삼성전자,삼성전자,20170814,20170814001832,,반기보고서 (2017.06)
4,005930,Y,삼성전자,삼성전자,20170515,20170515003806,,분기보고서 (2017.03)


보고서 보기
- 조회 결과에서 중요한 것은 문서의 고유 번호인 접수 번호 "rcp_no":"20170515003806" (삼성전자 분기보고서 2017.03)
- 이 번호를 rcpNo 인자 다음 URL을 만들어 문서를 조회할 수 있다
- In desktop : http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170515003806

In [24]:
df = result[:10] # 처음 10개만

for ix, row in df.iterrows():
    url_tmpl = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}"
    url = url_tmpl.format(row['rcp_no'])
    print (ix, url)

0 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180515001699
1 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402005019
2 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171114001694
3 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170814001832
4 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170515003806
5 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331004518
6 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20161114001491
7 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160816001779
8 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160516003174
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330003536


다음 자료에서는 개별 보고서의 하위 문서를 크롤링하는 것을 다루겠습니다.

이전에 했던 내용들을 복습합니다.

In [25]:
auth = '***'

In [27]:
# 당일 접수 10건
url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}'
url = url_tmpl.format(auth=auth)
r= requests.get(url)
jo = json.loads(r.text)
result = json_normalize(jo,'list')
result.head()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,002210,Y,동성제약,동성제약,20180618,20180618800164,유,전환가액ㆍ신주인수권행사가액ㆍ교환가액의조정(안내공시)
1,017180,Y,명문제약,명문제약,20180618,20180618000104,연,[기재정정]사업보고서 (2017.12)
2,267250,Y,현대중공업지주,윤중근,20180618,20180618000103,,임원ㆍ주요주주특정증권등소유상황보고서
3,267250,Y,현대중공업지주,현대중공업지주,20180618,20180618800161,유,최대주주등소유주식변동신고서
4,243070,K,휴온스,휴온스,20180618,20180618900160,코,기업설명회(IR)개최


In [30]:
#당일 접수 100건(max)
url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}&page_set=100'
url = url_tmpl.format(auth=auth)
r= requests.get(url)
r= json.loads(r.text)
result = json_normalize(r,'list')
result

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,002210,Y,동성제약,동성제약,20180618,20180618800164,유,전환가액ㆍ신주인수권행사가액ㆍ교환가액의조정(안내공시)
1,017180,Y,명문제약,명문제약,20180618,20180618000104,연,[기재정정]사업보고서 (2017.12)
2,267250,Y,현대중공업지주,윤중근,20180618,20180618000103,,임원ㆍ주요주주특정증권등소유상황보고서
3,267250,Y,현대중공업지주,현대중공업지주,20180618,20180618800161,유,최대주주등소유주식변동신고서
4,243070,K,휴온스,휴온스,20180618,20180618900160,코,기업설명회(IR)개최
5,006260,Y,LS,LS,20180618,20180618800159,유,타법인주식및출자증권취득결정(자회사의 주요경영사항)
6,005940,Y,NH투자증권,NH투자증권,20180618,20180618000096,,증권발행실적보고서
7,032800,K,판타지오,판타지오,20180618,20180618900155,코,임시주주총회결과
8,016360,Y,삼성증권,삼성증권,20180618,20180618000094,,증권발행실적보고서
9,056730,K,포스링크,포스링크,20180618,20180618900153,코,전환가액의조정(제16회차)


In [59]:
#회사의 전체 사업+반기+분기보고서 100건

url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003&page_set=100'
code = '005930'
url = url_tmpl.format(auth=auth,code=code)
r= requests.get(url)
jo = json.loads(r.text)
result = json_normalize(jo,'list')
result

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,005930,Y,삼성전자,삼성전자,20180515,20180515001699,,분기보고서 (2018.03)
1,005930,Y,삼성전자,삼성전자,20180402,20180402005019,연,사업보고서 (2017.12)
2,005930,Y,삼성전자,삼성전자,20171114,20171114001694,,분기보고서 (2017.09)
3,005930,Y,삼성전자,삼성전자,20170814,20170814001832,,반기보고서 (2017.06)
4,005930,Y,삼성전자,삼성전자,20170515,20170515003806,,분기보고서 (2017.03)
5,005930,Y,삼성전자,삼성전자,20170331,20170331004518,연,사업보고서 (2016.12)
6,005930,Y,삼성전자,삼성전자,20161114,20161114001491,,분기보고서 (2016.09)
7,005930,Y,삼성전자,삼성전자,20160816,20160816001779,,반기보고서 (2016.06)
8,005930,Y,삼성전자,삼성전자,20160516,20160516003174,,분기보고서 (2016.03)
9,005930,Y,삼성전자,삼성전자,20160330,20160330003536,연,사업보고서 (2015.12)


In [63]:
df =result[:10]

for ix,row in df.iterrows():
    url_tmpl = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}"
    url = url_tmpl.format(row['rcp_no'])
    print(ix,url)

0 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180515001699
1 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402005019
2 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171114001694
3 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170814001832
4 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170515003806
5 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331004518
6 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20161114001491
7 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160816001779
8 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160516003174
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330003536


In [64]:
import requests
import re

In [65]:
def get_report_doc_urls(rcp_no):
    doc_urls = []
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)
    reg = re.compile('viewDoc\((.*)\);')
    params = []
    matches = reg.findall(r.text)
    for m in matches:
        params.append(m.replace("'", "").replace(" ", "").split(","))
    doc_url_tmpl = "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 
    for p in params:
        if rcp_no == p[0]:
            doc_urls.append(doc_url_tmpl % tuple(p))
    return doc_urls

In [94]:
doc_urls = get_report_doc_urls('20170515003806')
for doc in doc_urls:
    print(doc)

http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=1&offset=564&length=4104&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=2&offset=12691&length=409&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=3&offset=13104&length=333222&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=4&offset=13194&length=234991&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=5&offset=248189&length=19537&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=6&offset=267730&length=4064&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=7&offset=271798&length=58143&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=8&offset=329945&length=7183&dtd=dart3.xsd
http://dart.fss.or.kr/rep

url which is used in code = http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=1&offset=564&length=4104&dtd=dart3.xsd
    

this code is in network>Doc>preview : "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 